<a href="https://colab.research.google.com/github/RojaCM/NLP/blob/main/fake%20classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.drop('id',axis=1,inplace=True)

In [11]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [12]:
df.shape

(18285, 4)

In [13]:
x=df.drop('label',axis=1)

In [14]:
y=df['label']

In [15]:
x.shape

(18285, 3)

In [16]:
y.shape

(18285,)

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.12.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [22]:
voc_size=5000

In [23]:
#One hot representation

In [35]:
messages=x.copy()

In [48]:
messages.reset_index(inplace=True)

In [49]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [50]:
messages.head(10)

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [51]:
import nltk
import re
from nltk.corpus import stopwords

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [54]:
len(messages)

18285

In [55]:
messages['title'][6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [60]:
corpus=[]

for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [63]:
corpus[10]

'russian research discov secret nazi militari base treasur hunter arctic photo'

In [64]:
onehot_repr=[one_hot(word,voc_size) for word in corpus]

In [65]:
onehot_repr[10]

[2897, 3471, 2518, 4403, 96, 3076, 3132, 1560, 1614, 4995, 1991]

In [71]:
sentence_len=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sentence_len)

In [72]:
from IPython.terminal.embed import EmbeddedMagics
print(embedded_docs[10])

[   0    0    0    0    0    0    0    0    0 2897 3471 2518 4403   96
 3076 3132 1560 1614 4995 1991]


In [69]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
len(embedded_docs)

18285

In [83]:
embedded_docs.shape

(18285, 20)

In [74]:
len(y)

18285

In [75]:
import numpy as np

In [76]:
x_data=np.array(embedded_docs)

In [77]:
y_data=np.array(y)

In [78]:
len(x_data)

18285

In [79]:
x_data.shape

(18285, 20)

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3,random_state=42)

In [82]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/5
200/200 [==============================] - 7s 27ms/step - loss: 0.3262 - accuracy: 0.8583 - val_loss: 0.2110 - val_accuracy: 0.9112
Epoch 2/5
200/200 [==============================] - 6s 31ms/step - loss: 0.1335 - accuracy: 0.9484 - val_loss: 0.1950 - val_accuracy: 0.9218
Epoch 3/5
200/200 [==============================] - 5s 23ms/step - loss: 0.0892 - accuracy: 0.9692 - val_loss: 0.2207 - val_accuracy: 0.9160
Epoch 4/5
200/200 [==============================] - 5s 27ms/step - loss: 0.0565 - accuracy: 0.9812 - val_loss: 0.2580 - val_accuracy: 0.9169
Epoch 5/5
200/200 [==============================] - 5s 23ms/step - loss: 0.0342 - accuracy: 0.9894 - val_loss: 0.3440 - val_accuracy: 0.9112


In [84]:
y_pred=model.predict(x_test)

172/172 [==============================] - 1s 4ms/step


In [85]:
y_pred=np.where(y_pred>0.5,1,0)

In [88]:
from sklearn.metrics import confusion_matrix

In [89]:
confusion_matrix(y_test,y_pred)

array([[2861,  246],
       [ 241, 2138]])

In [90]:
from sklearn.metrics import accuracy_score

In [91]:
accuracy_score(y_test,y_pred)

0.9112285818446956

In [92]:
from sklearn.metrics import classification_report

In [94]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3107
           1       0.90      0.90      0.90      2379

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486

